# 인공지능 학습용 데이터 객체 정보 수집 프로그램

## 작성정보

1. 작성일자 : 2021. 10. 24.

1. 작성자 : 한국지능정보사회진흥원 인공지능데이터전략팀 김민준 책임연구원

1. 버전정보 : ver.0.1

## 동작방법

1. AI 허브에서 다운로드 받은 인공지능 학습용 데이터 폴더를 선택한다.

1. 객체명이 기입된 정보를 입력한다.

1. 분석시작 구문을 실행한다.

1. 분석 결과를 확인하고, 엑셀로 출력해본다.

### 사전 정의 부

In [1]:
import os
import json
import pandas as pd
import xmltodict as xtd

# 객체명을 입력한다.
# 현재 class 라는 이름으로 입력되어 있고, 상황에 따라 따옴표 안에 내용을 변경한다.
# 예) class -> id
#class_key = 'category_id' # 주행 중 건물 촬영 이미지
class_key = 'name' # 도로주행영상
# 객체 통계 조사용 dict 구조
class_dict = {}

# 인공지능 학습용 데이터 폴더 선택
# ver0.1에서는 nb가 있는 폴더를 기본 폴더라고 생각
#data_dir = './'
#data_dir = 'D:/sidewalk polygon/주행 중 건물 촬영 이미지'
data_dir = 'D:/sidewalk polygon/도로주행영상'

# 함수 : 입력경로 내의 모든 하부 디렉토리 내의 json 파일을 조사하여 목록을 돌려줌
# 입력 : 입력경로
# 출력 : JSON 파일 리스트
def search_json(in_dir):
    json_list = []
    
    for (path, dir, files) in os.walk(in_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == '.json':
                json_list.append(f'{path}/{filename}')
                
    return json_list


# 함수 : 입력경로 내의 모든 하부 디렉토리 내의 xml 파일을 조사하여 목록을 돌려줌
# 입력 : 입력경로
# 출력 : xml 파일 리스트
def search_xml(in_dir):
    xml_list = []
    
    for (path, dir, files) in os.walk(in_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            if ext == '.xml':
                xml_list.append(f'{path}/{filename}')
                
    return xml_list


# 함수 : class_dict 내 value가 있으면 횟수증가, 없으면 1로 설정
# 입력 : value, dict
def counting_value(val):
    if val in class_dict:
        class_dict[val] = class_dict[val] + 1
    else:
        class_dict[val] = 1


# 함수 : json objects에서 특정 키 값의 횟수를 측정하여 dict에 기록
# 입력 : json object, key
def extract_key(json):
    if isinstance(json, dict):
        for k, v in json.items():
            if isinstance(v, (dict, list)):
                extract_key(v)
            elif k == class_key:
                counting_value(v)
    elif isinstance(json, list):
        for item in json:
            extract_key(item)

### 데이터 폴더 읽기

In [2]:
json_list = search_json(data_dir)
xml_list = search_xml(data_dir)

if not json_list and not xml_list:
    print('JSON 파일 또는 xml 파일이 없습니다. 폴더를 확인하고 다시 실행해주세요')
    
else:
    # json 경로정보 정제
    if json_list:
        json_list = [elem.replace('\\', '/') for elem in json_list]
    # xml 경로정보 정제
    if xml_list:
        xml_list = [elem.replace('\\', '/') for elem in xml_list]

    # json 경로정보 확인용
    '''
    for json_file in json_list:
        print(json_file)    
    '''
    
    # xml 경로정보 확인용
    for xml_file in xml_list:
        print(xml_file)    

D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000000_v001_1.xml
D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000030_v001_1.xml
D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000060_v001_1.xml
D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000090_v001_1.xml
D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000120_v001_1.xml
D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000150_v001_1.xml
D:/sidewalk polygon/도로주행영상/Training/bb/[라벨]도심로_야간일몰_맑음_30_전방/30_전방/20200708_200952/1_annotations_v001_1/1_20200708_200952_000180_v001_1.xml
D:/sidewalk polygon/

### 클래스 통계 조사

In [6]:
# json file list 조사
'''
for json_file in json_list:
    with open(json_file, 'r') as f:
        objects = json.load(f)
        extract_key(objects)
'''
        
# xml file list 조사
# TODO : 한글 인코딩 예외 처리
for xml_file in xml_list:
    with open(xml_file, 'r', encoding='utf-8') as f:
        objects = xtd.parse(f.read())
        extract_key(objects)
        
print(class_dict)

{'Vehicle_Car': 718, 'TrafficSign_Else': 126, 'TrafficSign_Speed': 18, 'TrafficLight_Green': 111, 'Vehicle_Bus': 8, 'Pedestrian_Pedestrian': 18, 'Vehicle_Unknown': 27, 'RoadMark_Character': 73, 'RoadMark_Number': 31, 'RoadMarkArrow_Straight': 94, 'TrafficLight_GreenArrow': 32, 'RoadMarkArrow_Left': 8, 'RoadMark_StopLine': 36, 'RoadMarkArrow_Else': 27, 'RoadMark_Crosswalk': 11, 'RoadMarkArrow_Right': 20, 'TrafficLight_Red': 95, 'Vehicle_Motorcycle': 2, 'RoadMarkArrow_StraightLeft': 8}


### 클래스 통계 내보내기

* 출력파일 : stat.csv

In [7]:
# Pandas의 DataFrame 활용
df = pd.DataFrame(data=class_dict, index=[0])
df = (df.T)
df.to_excel('stat.xlsx')